In [3]:
val path = "/Users/shawnmeier/Documents/source/bounder/soot_hopper/target/scala-2.13/soot_hopper-assembly-0.1.jar"

interp.load.cp(os.Path(path))

import $ivy.`org.plotly-scala::plotly-almond:0.8.1`
import $ivy.`com.lihaoyi:ujson_2.13:1.3.8`
// import $ivy.`edu.colorado.plv.bounder:soot_hopper_2.13:0.1`

import ujson.Value
import sys.process._

// interp.load.ivy("com.github.pathikrit" %% "better-files" % "3.9.1")
import $ivy.`com.github.pathikrit::better-files:3.9.1`
import better.files._



path: String = "/Users/shawnmeier/Documents/source/bounder/soot_hopper/target/scala-2.13/soot_hopper-assembly-0.1.jar"
import $ivy.$                                      

import $ivy.$                             
// import $ivy.`edu.colorado.plv.bounder:soot_hopper_2.13:0.1`


import ujson.Value

import sys.process._

// interp.load.ivy("com.github.pathikrit" %% "better-files" % "3.9.1")

import $ivy.$                                         

import better.files._



In [4]:
import scala.util.Random
import edu.colorado.plv.bounder.Driver
import edu.colorado.plv.bounder.RunConfig
import upickle.default.read
import scala.collection.parallel.CollectionConverters.{ImmutableSetIsParallelizable, IterableIsParallelizable}

import scala.util.Random

import edu.colorado.plv.bounder.Driver

import edu.colorado.plv.bounder.RunConfig

import upickle.default.read

import scala.collection.parallel.CollectionConverters.{ImmutableSetIsParallelizable, IterableIsParallelizable}

In [5]:
import edu.colorado.plv.bounder.symbolicexecutor.state.DBOutputMode
import edu.colorado.plv.bounder.symbolicexecutor.state.DBPathNode
import edu.colorado.plv.bounder.ir._
def obsMessagesStr(nodes:List[DBPathNode]):List[DBPathNode] = {
nodes.flatMap{n =>
  n.qry.loc match {
    case c @ CallinMethodReturn(fmwClazz, fmwName) => Some(n)
    case c @ CallinMethodInvoke(fmwClazz, fmwName) => Some(n)
    case c @ GroupedCallinMethodInvoke(targetClasses, fmwName) => Some(n)
    case c @ GroupedCallinMethodReturn(targetClasses, fmwName) => Some(n)
    case c @ CallbackMethodInvoke(fmwClazz, fmwName, loc) => Some(n)
    case c @ CallbackMethodReturn(fmwClazz, fmwName, loc, line) => Some(n)
    case _ => None
  }}
}
// val idents = List("net.khertan.forrunners/sample3")
val idents = List("com.ridgelineapps.resdicegame/sample3", "com.ridgelineapps.resdicegame/sample1")
val basedir = "/Users/shawnmeier/Documents/source/bounder/experiments/fdroid_subset_5"
def dbgInfo(ident:String){
    val db = DBOutputMode(s"${basedir}/${ident}/paths.db")
    val live = db.liveTraces()
    println(s"live locations: ${live.size}")
    val traceSet = live.map(tr => obsMessagesStr(tr)).toSet
    println(s"unique obs traces: ${traceSet.size}")
    // Write sample trace to file
    val out = File(s"${basedir}/${ident}")
    val log = out / "obsLog.txt"
    if(log.exists()) log.delete()
    traceSet.foreach{ tr1 =>
        val tr1 = traceSet.head
        log.append("======\n")
        tr1.foreach{n => 
            log.append(n.qry.loc.toString)
            log.append("\n")
            log.append(n.qry.state.toString)
            log.append("\n")
        }
    }
        
    // Count distinct locations in wit
    val locLog = out / "locLog.txt"
    if(locLog.exists()) locLog.delete()
    val locs = live.flatten.groupBy(n => n.qry.loc.containingMethod).map{case (k,v) => (v.size,k)}.toList.sortBy(_._1)
    locs.foreach(l => locLog.append(s"${l._1} , ${l._2.toString}\n"))
        
    // write stack of sample
    val stackLog = out/ "stackLog.txt"
    if(stackLog.exists()) stackLog.delete()
    val pathNode = live.head.head
    pathNode.qry.state.callStack.foreach(v => stackLog.append(v.exitLoc.toString + "\n"))
    stackLog.append(pathNode.qry.state.toString)
}
idents.foreach(ident => dbgInfo(ident))

[scala-interpreter-1] WARN slick.util.AsyncExecutor - Having maxConnection > maxThreads can result in deadlocks if transactions or database locks are used.


: 

In [8]:
import plotly._
import plotly.element._
import plotly.layout._
import plotly.Almond._

val (x, y) = Seq(
  "Banana" -> 10,
  "Apple" -> 8,
  "Grapefruit" -> 5
).unzip

Scatter(x, y).plot()

import plotly._

import plotly.element._

import plotly.layout._

import plotly.Almond._


x: Seq[String] = List("Banana", "Apple", "Grapefruit")
y: Seq[Int] = List(10, 8, 5)
res7_5: String = "plot-b3e0cba7-4d54-438a-9083-91b55f7f045d"